## Preliminary Functions

Encoding and decoding function that converts a string into an integer. 

In [ ]:
# Function to encode a string into an integer
def encode_message(m):
    # Convert each character to an 8-bit binary string
    bin_str = ''.join(format(ord(c), '08b') for c in m)
    
    # Convert the binary string into an integer
    z = int(bin_str, 2)
    return z


# Function to decode integers into a string
def decode_message(z):
    # Convert the integer to a binary string
    bin_str = bin(z)[2:]  # Strip the '0b' prefix
    
    # Ensure the binary string length is a multiple of 8
    padding = len(bin_str) % 8
    if padding != 0:
        bin_str = '0' * (8 - padding) + bin_str
    
    # Split the binary string into 8-bit chunks and convert them to characters
    decoded_message = ''.join([chr(int(bin_str[i:i + 8], 2)) for i in range(0, len(bin_str), 8)])
    
    return decoded_message

Functions that invert in modular arithmetic. 

In [ ]:
# Function to find the modular inverse of a number a modulo m
def Invert(a, m):
    x, y, gcd = extended_gcd(a, m)
    if gcd != 1:
        return None  # check for no modular inverses
    return x % m 

# Extended Euclidean Algorithm to find the greatest common divisor (GCD) and coefficients
def extended_gcd(a, b):
    if b == 0:
        return 1, 0, a # base case
    
    x1, y1, gcd = extended_gcd(b, a % b) # induction

    x, y = y1, x1 - (a // b) * y1 
    return x, y, gcd


Function for generating primes

In [32]:
# Function for generating primes
import sympy

def generate_prime(bit_length):
    """Generates a prime number with the specified bit length."""
     
    # Generate a random prime of the required bit length
    prime = sympy.randprime(2**(bit_length - 1), 2**bit_length)

    return prime

## Elliptic Curve Cryptography

Elliptic curve class

In [71]:
## Elliptic Curve Class

import secrets

class EC:
    def __init__(self, p, a, b):
        """ 
        Class that defines an elliptic curve over mod
        """

        if 4*pow(a,3)+27*pow(b,2) % p == 0:
            raise ValueError("Curve has zero discriminant")
        else:
            self.p = p
            self.a = a % p
            self.b = b % p

    def check(self, point):
        """ 
        Check point lies in the elliptic curve
        """
        x, y = point

        # Check to see if point is 'zero' 
        if point == [0,0]:
            return True

        # Calculate Y^2 and y^2
        self.Y2 = (pow(x,3) + self.a*x + self.b) % self.p
        self.y2 = pow(y,2) % self.p

        # Check to see if non zero point is in curve
        if  self.Y2 == self.y2:
            return True
        else:
            return False


    def add(self, point1, point2):
        """ 
        Add two points on the elliptic curve: R = P + Q 
        """
        # Check points are on the elliptic curve
        if self.check(point1) == False or self.check(point2) == False:
            raise ValueError("One of the points are not on the elliptic curve.")
        
        # Case where noe of the points is zero 
        if point1 == [0,0]:
            return point2
        elif point2 == [0,0]:
            return point1
        
        # Unpack coordinates
        x1, y1 = point1 
        x2, y2 = point2

        # Reduce coordinates in mod p
        x1 = x1 % self.p
        x2 = x2 % self.p
        y1 = y1 % self.p
        y2 = y2 % self.p

        # Check if points are negatives of each other
        if x1 == x2 and y1 == (-y2) % self.p:
            return [0, 0]
        else:
            if point1 == point2: # point doubling
                l1 = (3*pow(x1,2) + self.a) % self.p
                l2 = (2*y1) % self.p
                l = (l1*Invert(l2, self.p)) % self.p # calculate slope
            else: # point addition
                l1 = (y2-y1) % self.p
                l2 = (x2-x1) % self.p
                l = (l1*Invert(l2,self.p)) % self.p # calculate slope
    
        # Calculate the new coordinates
        x3 = (pow(l,2)-x1-x2) % self.p
        y3 = (l*(x1-x3) - y1) % self.p

        return [x3, y3]
    
    def negate(self, point):
        """ Negative multipliation on the elliptic curve (reflect over x-axis) """
        x, y = point
        return [x, (-y) % self.p]

    def subtract(self, point1, point2):
        """ Subtract two points on the elliptic curve: P - Q = P + (-Q) """
        # Negate point2
        neg_point2 = self.negate(point2)
        # Add point1 and the negation of point2
        return self.add(point1, neg_point2)


    def double_add(self, point, n):
        """
        Double and add method for scalar multiplication on elliptic curves.
        """
        self.Q = point
        self.R = [0, 0] 

        while n > 0:
            if n % 2 == 1:  # If n is odd, add the current point to R
                self.R = self.add(self.R, self.Q)
            
            # Double the point Q
            self.Q = self.add(self.Q, self.Q)
            
            # Divide n by 2 (integer division)
            n = n // 2
        
        return self.R
    
    
            
    def DF_key_exchange(self, Point, key):
        """ 
        Function for key exchange
        """
        return self.double_add(Point, key)
    

Elgamal Class

In [72]:
import secrets

class ElGamal:
    def __init__(self, prime, coeff_a, coeff_b, Point):
        # Create elliptic curve object
        self.curve = EC(prime, coeff_a, coeff_b)

        if self.curve.check(Point):
            self.P = Point
        else:
            raise ValueError("Point chosen is not on specified elliptic curve.")

    def key(self, private_key):
        """
        Generates the public key corresponding to the given private key.
        """
        self.private_key = private_key
        self.public_key = self.curve.double_add(self.P, self.private_key)
        return self.public_key

    def encrypt(self, public_key, plaintext):
        """
        Encrypts a plaintext message using the public key.
        """
        # Generate a random ephemeral key k
        self.k = secrets.randbelow(self.curve.p)

        # Calculate cipher texts
        C1 = self.curve.double_add(self.P, self.k)
        C2 = self.curve.add(plaintext, self.curve.double_add(public_key, self.k))

        return [C1, C2]

    def decrypt(self, ciphertext):
        """
        Decrypts the ciphertext using the private key.
        """
        C1, C2 = ciphertext
        # Calculate d * C1, then subtract from C2 to get the plaintext
        return self.curve.subtract(C2, self.curve.double_add(C1, self.private_key))


Key Exchange example

In [75]:
## Diffie-Hellman key exchange 

# Choose public parameters 
p = 13
a = 3
b = 8
P = [1,8]

# Create elliptic curves
Alice = EC(p,a,b)
Bob = EC(p,a,b)

# Alice chooses a secret key and creates a public key
A = Alice.DF_key_exchange(P,10)

# Bob chooses a secret key and creates a public key
B = Bob.DF_key_exchange(P,8)

# Both parties calculate the shared key
key_A = Alice.DF_key_exchange(B,10)
key_B = Bob.DF_key_exchange(A,8)

key_A == key_B

True

Elgamal example

In [78]:
## Elgamal Elliptic Curve

# Choose public parameters 
p = 13
a = 3
b = 8
P = [1,8]

# Create elliptic curves
Alice = ElGamal(p,a,b,P)
Bob = ElGamal(p,a,b,P)

# Alice chooses a private key and generates a public key
key = Alice.key(2)

# Bob encodes a point using Alice's public key
cipher_text = Bob.encrypt(key,[12,11])

# Alice decrypts Bob's cypher text
Alice.decrypt(cipher_text)


[12, 11]